<a href="https://colab.research.google.com/github/stepbot/StableDiffusionNotebooks/blob/main/singleImageCannyControlNetWithGFPGANpost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone GFPGAN and enter the GFPGAN folder
%cd /content
!rm -rf GFPGAN
!git clone https://github.com/TencentARC/GFPGAN.git
%cd GFPGAN

# Set up the environment
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr
# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib
# Install other depencencies
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan  # used for enhancing the background (non-face) regions
# Download the pre-trained model
# !wget https://github.com/TencentARC/GFPGAN/releases/download/v0.2.0/GFPGANCleanv1-NoCE-C2.pth -P experiments/pretrained_models
# Now we use the V1.4 model for the demo
!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -P experiments/pretrained_models
%cd /content

In [ ]:
!pip install diffusers[torch]==0.14.0 transformers scipy ftfy accelerate
!pip install -q opencv-contrib-python
!pip install -q controlnet_aux

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers.utils import load_image
import torch
import cv2
from PIL import Image
import numpy as np

In [ ]:
# select you model here
model_id = "dreamlike-art/dreamlike-photoreal-2.0"

controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    model_id, controlnet=controlnet, torch_dtype=torch.float16
)

In [ ]:
# load input image
try:
  input_image=Image.open('input.png')
except:
  input_image = load_image(
    "https://hf.co/datasets/huggingface/documentation-images/resolve/main/diffusers/input_image_vermeer.png"
)
input_image

In [ ]:
image = np.array(input_image)

low_threshold = 100
high_threshold = 200

image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)
canny_image.save("canny.png")
canny_image

In [ ]:
pipe.safety_checker=None
pipe = pipe.to("cuda")

# set seed
generator = torch.Generator("cuda").manual_seed(42)

# set prompts
prompt = "Nicolas Cage"
n_prompt = "worst quality,monochrome, bad anatomy, low resolution, extra limbs, unnatural, messed up body, extra fingers"

# generate an image
output = pipe(prompt,canny_image, negative_prompt=n_prompt, guidance_scale=8, num_inference_steps=50, height=768, width=768).images[0]
output.save("output.png")
output



In [ ]:
# Now we use the GFPGAN to restore the above low-quality images
# We use [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN) for enhancing the background (non-face) regions
# You can find the different models in https://github.com/TencentARC/GFPGAN#european_castle-model-zoo
!rm -rf postprocessed
!python /content/GFPGAN/inference_gfpgan.py -i "output.png" -o postprocessed -v 1.4 -s 2 --bg_upsampler realesrgan
post_image=Image.open('/content/postprocessed/restored_imgs/output.png')
post_image
# Usage: python inference_gfpgan.py -i inputs/whole_imgs -o results -v 1.3 -s 2 [options]...
# 
#  -h                   show this help
#  -i input             Input image or folder. Default: inputs/whole_imgs
#  -o output            Output folder. Default: results
#  -v version           GFPGAN model version. Option: 1 | 1.2 | 1.3. Default: 1.3
#  -s upscale           The final upsampling scale of the image. Default: 2
#  -bg_upsampler        background upsampler. Default: realesrgan
#  -bg_tile             Tile size for background sampler, 0 for no tile during testing. Default: 400
#  -suffix              Suffix of the restored faces
#  -only_center_face    Only restore the center face
#  -aligned             Input are aligned faces
#  -ext                 Image extension. Options: auto | jpg | png, auto means using the same extension as inputs. Default: auto